In [1]:
!pip install fastai

   ---------------------------------------- 0.0/11.7 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.7 MB 10.0 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.7 MB 10.9 MB/s eta 0:00:01
   ------------------------ --------------- 7.1/11.7 MB 11.8 MB/s eta 0:00:01
   ---------------------------------- ----- 10.2/11.7 MB 12.3 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.7 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------- 11.7/11.7 MB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.5 MB 8.4 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/11.5 MB 8.4 MB/s eta 0:00:01
   ------------------ --------------------- 5.2/11.5 MB 8.6 MB/s eta 0:00:01
   ------------------------ --------------- 7.1/11.5 MB 8.7 MB/s eta 0:00:01
   ------------------------------- -------- 8.9/11.5 MB 8.8 MB/s eta 0:00:01
   -

In [5]:
from fastai.vision.all import *
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import random

In [22]:
data_dir = 'D:/processed_plantas'
categories = ['arocarbol', 'durazno', 'guayabilla', 'matico', 'tuna']
img_paths = []
labels = []

images_per_category = 5000

# Recolección de imágenes y etiquetas
for label, category in enumerate(categories):
    category_path = os.path.join(data_dir, category)
    img_names = os.listdir(category_path)
    random.shuffle(img_names) 
    img_names = img_names[:images_per_category] 
    img_paths += [os.path.join(category_path, img_name) for img_name in img_names]
    labels += [label] * len(img_names)

In [23]:
# División en conjunto de entrenamiento y prueba
train_paths, test_paths, train_labels, test_labels = train_test_split(img_paths, labels, test_size=0.2, stratify=labels)

# Crear un DataFrame para los datos de entrenamiento y prueba
df_train = pd.DataFrame({'image': train_paths, 'label': train_labels})
df_test = pd.DataFrame({'image': test_paths, 'label': test_labels})

In [24]:
# Convertir la columna 'image' a ruta de archivos
def get_x(row): return row['image']
def get_y(row): return row['label']

# Creación del DataBlock de Fastai
plant_data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=get_x,
    get_y=get_y,
    splitter=RandomSplitter(valid_pct=0.2),
    batch_tfms=Normalize.from_stats(*imagenet_stats)
)

# Crear el DataLoader
dls = plant_data.dataloaders(df_train, bs=32)

In [25]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(300*300*3, 50)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(50, 5)

    def forward(self, x):
        x = x.view(-1, 300*300*3)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [26]:
# Instanciar el modelo y utilizar GPU si está disponible
model = MLP()

# Crear el learner
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

print(f'Model is on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}')

Model is on: NVIDIA GeForce GTX 1060


In [27]:
# Entrenamiento del modelo
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.019946,4.503966,0.321000,01:47
1,2.846083,2.363024,0.348250,01:34
2,1.863835,1.706053,0.375000,01:33
3,1.894662,1.689463,0.377750,01:34
4,1.678151,1.715677,0.441750,01:36
5,1.587153,1.481910,0.447500,01:34
6,1.424334,1.367474,0.472750,01:35
7,1.272260,1.253010,0.510000,01:32
8,1.152949,1.219603,0.533250,01:32
9,1.110680,1.213248,0.532750,01:34


In [28]:
# Guardar modelo entrenado
learn.save('mlp-fastai')

Path('models/mlp-fastai.pth')

In [29]:
# Evaluación en el set de validación
learn.validate(dl=dls.valid)

(#2) [1.2132481336593628,0.5327500104904175]

In [ ]:
# Concluciones
# - El modelo muestra una tendencia positiva en la reduccion de la perdida y 
#   aumento de la precision a lo largo del entrenamiento.
# - La perdida de validadcion ha mostrado algo de fluctuacion, lo cual pude ser normal, pero tambien podria ser indicativo
#   que el modelo podria beneficiarse de un ajuste mas fino o una mayor regularizacion.
# - La precision final de 53.3% indica que el modelo esta en un punto medio. Hay margen para mejoras adicionales. 